In [1]:
from oef.agents import Agent
import asyncio
from typing import List

from oef.agents import Agent
from oef.proxy import OEFNetworkProxy
from oef.query import Query
from oef.schema import DataModel, Description

In [2]:
class GreetingsAgent(Agent):
    """A class that implements the greeting agent."""

    def on_message(self, origin: str, dialogue_id: int, content: bytes):
        print("[{}]: Received message: origin={}, dialogue_id={}, content={}"
              .format(self.public_key, origin, dialogue_id, content))
        if content == b"hello":
            print("[{}]: Sending greetings message to {}".format(self.public_key, origin))
            self.send_message(dialogue_id, origin, b"greetings")
            self.stop()
        if content == b"greetings":
            print("[{}]: Received greetings message from {}".format(self.public_key, origin))
            self.stop()

    def on_search_result(self, search_id: int, agents: List[str]):
        if len(agents) > 0:
            print("[{}]: Agents found: {}".format(self.public_key, agents))
            for a in agents:
                self.send_message(0, a, b"hello")
        else:
            print("[{}]: No agent found.".format(self.public_key))


In [3]:
# create agents

client_proxy = OEFNetworkProxy("greetings_client", oef_addr="127.0.0.1", port=3333)
server_proxy = OEFNetworkProxy("greetings_server", oef_addr="127.0.0.1", port=3333)
client_agent = GreetingsAgent(client_proxy)
server_agent = GreetingsAgent(server_proxy)

In [4]:
# connect the agents to the OEF
await client_agent.async_connect()
await server_agent.async_connect()

In [5]:
# register the greetings service agent on the OEF
greetings_model = DataModel("greetings", [], "Greetings service.")
greetings_description = Description({}, greetings_model)
server_agent.register_service(greetings_description)

In [6]:
# the client executes the search for greetings services
query = Query([], greetings_model)

print("[{}]: Search for 'greetings' services.".format(client_agent.public_key))
client_agent.search_services(0, query)


[greetings_client]: Search for 'greetings' services.


In [7]:
# run the agents
try:
    await asyncio.gather(
                client_agent.async_run(),
                server_agent.async_run())
finally:
    # if something bad happens, stop the agents 
    client_agent.stop()
    server_agent.stop()



[greetings_client]: Agents found: ['greetings_server']
[greetings_server]: Received message: origin=greetings_client, dialogue_id=0, content=b'hello'
[greetings_server]: Sending greetings message to greetings_client
[greetings_client]: Received message: origin=greetings_server, dialogue_id=0, content=b'greetings'
[greetings_client]: Received greetings message from greetings_server
